In [2]:
using Distributions
using LinearAlgebra
using Plots
using Sobol
using Random
using ForwardDiff

include("../testfns.jl")
include("../utils.jl")
include("../rollout.jl")

simulate_trajectory (generic function with 1 method)

## Construct the acquisition function
Our acquisition functions directly influence our choice of sample location. They can be thought of as query strategies. The most common acquisition functions in the vanilla Bayesian optimization literature are *Probability of Improvement, Expected Improvement, and Upper Confidence Bound*. The derivation of the acquisition functions under the Gaussian process posterior in 1D is analytic and given as follows.
<hr>

#### Probability of Improvement
The probability of improvement (POI) is defined as follows:

$$
POI(x) = \Phi\left( \frac{f^+ - \mu(x)}{\sigma(x)} \right)
$$

where $f^+$ denotes the best value (maximum) known, $\mu(x)$ is the predictive mean at x, $\sigma(x)$ is the predictive variance, $\xi$ is our exploration parameter, and $\Phi$ is the standard normal cumulative distribution function.
<hr>

#### Expected Improvement
The expected improvement (EI) is defined as follows:

$$
EI(x) = (f^+ - \mu(x))\Phi\left( \frac{f^+ - \mu(x)}{\sigma(x)} \right) +
        \sigma(x)\phi\left(\frac{f^+ - \mu(x)}{\sigma(x)}\right)
$$

where $\phi$ is the standard normal probability density function.
<hr>

#### Upper Confidence Bound
The upper confidence bound (UCB) is defined as follows:

$$
UCB(x) = \mu(x) + \beta\sigma(x)
$$

where $\beta$ is our exploration parameter.
<hr>

In [ ]:
# z(μ, σ, f⁺; ξ) = (μ-f⁺-ξ)/σ
z(μ, σ, f⁺; ξ) = (f⁺-μ-ξ)/σ # we're minimizing objectives here

function POI(s::RBFsurrogate; ξ=1., tol=1e-6)
    f⁺ = minimum(s.y)
    return function POIx(x::Float64)
        sx = s([x])
        if sx.σ <= tol return 0 end
        
        Φx = cdf(Normal(), z(sx.μ, sx.σ, f⁺; ξ=ξ))
        
        return Φx
    end
end

function EI(s::RBFsurrogate; ξ=1., tol=1e-6)
    f⁺ = minimum(s.y)
    return function EIx(x::Float64)
        sx = s([x])
        if sx.σ <= tol return 0 end

        zx = z(sx.μ, sx.σ, f⁺; ξ=ξ)
        Φx = cdf(Normal(), zx)
        ϕx = pdf(Normal(), zx)
        
        return (zx*sx.σ)*Φx + sx.σ*ϕx
    end
end

function UCB(s::RBFsurrogate; β=1., tol=1e-6)
    return function UCBx(x::Float64)
        sx = s([x])
        if sx.σ <= tol return end
        
        return sx.μ + β*sx.σ
    end
end

function stdize(series)
    smax, smin = maximum(series), minimum(series)
    return [(s-smin)/(smax-smin) for s in series]
end